Ноутбук создан для выполнения отборочного задания Т-Банка и Сириуса

In [36]:
!pip -q install -U "bitsandbytes==0.43.3" "transformers>=4.44.0" "accelerate>=0.34.0" "peft>=0.11.0"
import torch, bitsandbytes as bnb
print("CUDA in torch:", torch.version.cuda)
print("bitsandbytes:", bnb.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.3 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'triton.ops'

In [1]:
!pip -q install -U transformers accelerate peft bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.6 MB/s eta 0:00:00


In [2]:
import os, math, json, random, re
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Tuple
import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig
)


In [3]:
DATA_DIR = "/content"
TRAIN_CSV = f"{DATA_DIR}/train.csv"
CATS_TXT  = f"{DATA_DIR}/categories.txt"

In [4]:
def read_categories(path: str) -> List[str]:
    with open(path, "r", encoding="utf-8") as f:
        cats = [line.strip() for line in f if line.strip()]
    return cats

def load_train(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    if "text" not in df.columns:
        raise ValueError("Ожидалась колонка 'text' в train.csv")
    return df

In [5]:
categories  = read_categories(CATS_TXT)

In [6]:
train_df = load_train(TRAIN_CSV)

Для начало сделаем автолейблинг с помощью Qwen

In [7]:
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
LOAD_IN_4BIT = True

In [8]:
MAX_PROMPT_TOKENS = 1024
N_REPHRASES = 3

# Порог уверенности для принятия псевдо-лейбла
CONFIDENCE_THRESHOLD = 0.35

In [9]:
OUT_CSV = f"{DATA_DIR}//pseudo_labeled_llm.csv"

In [10]:
REPHRASE_TEMPLATES = [
    ("Тебе дан список категорий товаров. Нужно выбрать ровно ОДНУ категорию, "
     "которой соответствует отзыв. Если отзыв не про товар, выбери «нет товара»."),
    ("Определи, к какой категории относится отзыв. Верни строго одно имя класса из списка. "
     "Если отзыв лишь про сервис/доставку — «нет товара»."),
    ("Классифицируй отзыв по списку категорий. Ответ должен быть только названием класса. "
     "Для отзывов без товара выбери «нет товара»."),
]

In [11]:
def build_prompt(txt: str, cats: List[str], variant: int = 0) -> str:
    head = REPHRASE_TEMPLATES[variant % len(REPHRASE_TEMPLATES)]
    return (
        f"{head}\n\n"
        f"Категории: {', '.join(cats)}\n\n"
        f"Отзыв: {txt}\n"
        f"Класс:"
    )

In [12]:
def load_llm(name: str, load_in_4bit=True):
    bnb = BitsAndBytesConfig(
        load_in_4bit=load_in_4bit,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    ) if load_in_4bit else None

    tok = AutoTokenizer.from_pretrained(name, use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        name,
        device_map="auto",
        quantization_config=bnb,
        torch_dtype=torch.bfloat16 if load_in_4bit else torch.float16
    )
    model.eval()
    return tok, model

In [13]:

tokenizer, model = load_llm(MODEL_NAME, LOAD_IN_4BIT)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [14]:
@torch.no_grad()
def class_score_logprob(prompt: str, label: str) -> float:
    """
    Teacher-forcing: считаем сумму логвероятностей токенов метки `label`,
    если модель "продолжает" prompt этой меткой.
    """
    # Формируем input = prompt + label
    target = label.strip()
    full = prompt + " " + target
    enc = tokenizer(full, return_tensors="pt", truncation=True, max_length=MAX_PROMPT_TOKENS).to(model.device)

    # Индексы целевых токенов (метка после prompt)
    input_ids = enc["input_ids"][0]
    # Найдём, где начинаются токены label
    lab_ids = tokenizer(" " + target, add_special_tokens=False, return_tensors="pt")["input_ids"][0]
    L = lab_ids.shape[-1]
    # позиция начала label = последние L токенов
    start = input_ids.shape[-1] - L
    assert start >= 1, "label слишком длинная или prompt усечён"

    # Прямой прогон
    out = model(input_ids=input_ids.unsqueeze(0))
    logits = out.logits[0]  # (seq_len, vocab)

    # log p(label_tokens | prompt) = сумма по i: log softmax(logits[start-1+i])[lab_id[i]]
    log_probs = torch.log_softmax(logits[start-1:start-1+L], dim=-1)  # (L, vocab)
    token_logps = log_probs[torch.arange(L), lab_ids.to(model.device)]
    return float(token_logps.sum().item())

In [15]:
@torch.no_grad()
def classify_zero_shot(text: str, cats: List[str], n_rephrases: int = 3) -> (str, float, Dict[str, float]):
    """
    Возвращает: (best_label, best_prob, probs_dict)
    где probs_dict — усреднённые по переформулировкам вероятности категорий (softmax).
    """
    # Подготовим 1–3 версии промпта
    txt = re.sub(r"\s+", " ", str(text)).strip()
    if len(txt) > 700:
        txt = txt[:700] + "…"

    # Накапливаем суммарные logp по категориям (агрегация по переформулировкам)
    agg_logps = np.zeros(len(cats), dtype=np.float64)

    for v in range(n_rephrases):
        prompt = build_prompt(txt, cats, variant=v)
        # посчитаем логскор каждой категории
        logs = []
        for c in cats:
            lp = class_score_logprob(prompt, c)
            logs.append(lp)
        # нормализация внутри промпта (log-softmax)
        m = np.max(logs)
        probs = np.exp(np.array(logs) - m)
        probs = probs / probs.sum()
        # аккумулируем в лог-пространстве как суммы log(prob)
        agg_logps += np.log(np.clip(probs, 1e-12, 1.0))

    # финальные вероятности = softmax от агрегированных логов
    m2 = np.max(agg_logps)
    final_probs = np.exp(agg_logps - m2)
    final_probs = final_probs / final_probs.sum()

    best_idx = int(np.argmax(final_probs))
    best_label = cats[best_idx]
    best_prob = float(final_probs[best_idx])

    probs_dict = {c: float(final_probs[i]) for i, c in enumerate(cats)}
    return best_label, best_prob, probs_dict

In [ ]:
preds, probs = [], []
for i, row in train_df.iterrows():
    label, p, _ = classify_zero_shot(row["text"], categories, n_rephrases=N_REPHRASES)
    preds.append(label)
    probs.append(p)
    if (i) % 100 == 0:
        print(f"[{i+1}/{len(train_df)}] -> {label} (p={p:.3f})")

[1/1818] -> одежда (p=1.000)


In [ ]:
out = train_df.copy()
out["category"] = preds
out["prob"] = probs
out.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)

In [ ]:
#распределение
print(out["category"].value_counts())